### Description

- This code merges csv files into one csv file for each station
- Source data structure: Station(folder) -> Substance(subfolder) -> Data for a certain period (CSV files)
- Output data strcture: Combined CSV files for each station

In [1]:
# Import Libraries
import pandas as pd
import os
import glob
import re

In [2]:
# Get folder list
data_dir = r'Source'
sub_folders = os.listdir(data_dir)
sub_folders

['Central San Bernardino',
 'Temecula',
 'Central LA',
 '.DS_Store',
 'Mira Loma',
 'Reseda',
 'Upland',
 'Santa Clarita',
 'Banning',
 'Lake Elsinore',
 'Glendora']

In [4]:
# For each folder, combine the csv files into one csv file
for station in sub_folders:
    # If folder name is '.DS_Store', skip the loop
    if station in ['.DS_Store', 'csv_file_merger']:
        continue
    
    # Get a list of subfolders for each folder (or each station)
    station_path = os.path.join(data_dir, station)
    materials = os.listdir(station_path)
    
    # Flag variable for checking whether current loop is the first one
    cnt2 = 0
    
    # Loop over subfolders in each folder
    for material in materials:
        # Skip the loop if subfolder name is '.DS_Store' or 'csv_file_merger'
        if station in ['.DS_Store', 'csv_file_merger']:
            continue
        
        # Get a list of csv files for each subfolder (or substance)
        material_path = os.path.join(station_path, material)
        all_files = glob.glob(material_path + '/*.csv')

        if len(all_files)==0:
            continue
        
        # Flag variable for checking whether current loop is the first one
        cnt = 0
        
        # Loop over csv files for each subfolder and merge csv files into one csv file for each substance (or subfolder)
        for filename in all_files:
            if cnt == 0:
                df = pd.read_csv(filename, on_bad_lines='skip').iloc[:, 0:3].rename(columns={"Value": material})
            else:
                df_new = pd.read_csv(filename, on_bad_lines='skip').iloc[:, 0:3].rename(columns={"Value": material})
                df = pd.concat([df, df_new], axis=0)
            cnt+=1
        
        # Merge csv files into one csv file for each station (or folder)
        if cnt2 == 0:
            df_st = df
        else:
            df_st=pd.merge(df_st, df, on=['Date Time', 'Site Name'])
    
        cnt2+=1
    
    # Save final results
    df_st['Date Time'] = pd.to_datetime(df_st['Date Time'])
    df_st = df_st.sort_values(by='Date Time')
    df_st.to_csv(data_dir + '/output/' + station.replace(' ', '_') + '.csv', index=False)